<a href="https://colab.research.google.com/github/bundickm/Automated_RSS_Feed_Summarizer/blob/master/RSS_Feed_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull From RSS Feed

In [1]:
!pip install feedparser

     |████████████████████████████████| 194kB 11.1MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=e4701ad1d82a6536fe5d6e4b77cf01254544f4bf753795da0de4ce34f46692f4
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser


In [0]:
import feedparser # pip install feedparser

reader = feedparser.parse('http://rss.cnn.com/rss/cnn_topstories.rss')
# .. skipped handling http errors, cacheing ..

parsed_feed = []
for entry in reader.entries:
  parsed_feed.append({
      'title': entry.title,
      'url': entry.link,
      'description': entry.description})  

In [3]:
parsed_feed

[{'description': 'President Donald Trump on Tuesday denied that he directed his personal lawyer Rudy Giuliani to go to Ukraine and seek out investigations on his behalf, contradicting his own words to the Ukrainian President in the White House released transcript of the July 25 call.<div class="feedflare">\n<a href="http://rss.cnn.com/~ff/rss/cnn_topstories?a=aVb9k7UPBOk:-JYAI8cuhO8:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/rss/cnn_topstories?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://rss.cnn.com/~ff/rss/cnn_topstories?a=aVb9k7UPBOk:-JYAI8cuhO8:7Q72WNTAKBA"><img src="http://feeds.feedburner.com/~ff/rss/cnn_topstories?d=7Q72WNTAKBA" border="0"></img></a> <a href="http://rss.cnn.com/~ff/rss/cnn_topstories?a=aVb9k7UPBOk:-JYAI8cuhO8:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/rss/cnn_topstories?i=aVb9k7UPBOk:-JYAI8cuhO8:V_sGLiPBpWU" border="0"></img></a> <a href="http://rss.cnn.com/~ff/rss/cnn_topstories?a=aVb9k7UPBOk:-JYAI8cuhO8:qj6IDK7rITs"><img src="htt

# Parse Web Pages

In [0]:
from bs4 import BeautifulSoup
import requests

def get_text(url):
  html = requests.get(url)
  soup = BeautifulSoup(html.text)

  for script in soup(["script", "style"]):
      script.decompose()    # rip it out
  text = soup.find('section', {'id':'body-text'}).get_text()

  # get text
  # text = soup.get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  start = text.find('(CNN)')
  end = text.find('\nSearch')
  return text[start+5:end]

In [23]:
test = get_text('http://rss.cnn.com/~r/rss/cnn_topstories/~3/CO9DFp_HIA4/index.html')
test

'On a recent November evening, Bob Blackman scrolled through his phone in a nondescript Indian restaurant in west London, sparsely decorated except for stock pictures of East African sunsets and lions. He proudly pointed to his phone\'s background photo, which shows him smiling next to Indian Prime Minister Narendra Modi."I\'ve been to India seven times in the last three years," Blackman said, sipping a cup of chai, his cheeks flushed from canvassing in the cold, ahead of Britain\'s first December election in nearly a century. "I am well-known for being a very pro-Indian supporter."Across the table, Kuldeep Singh Shekhawat, the head of a UK-based pressure group linked to Modi\'s Bharatiya Janata Party (BJP), chimed in: "A hundred percent." Blackman is a three-term member of Parliament for Harrow East, a racially and religiously diverse constituency where 28.2% of the residents are Hindu. The Conservative Party politician has a track record of championing Indian -- more specifically, Hi

# Create Tags with LDA

In [0]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [24]:
test = test.replace('\n', ' ')
test = test.lower().split('.')
test

['on a recent november evening, bob blackman scrolled through his phone in a nondescript indian restaurant in west london, sparsely decorated except for stock pictures of east african sunsets and lions',
 " he proudly pointed to his phone's background photo, which shows him smiling next to indian prime minister narendra modi",
 '"i\'ve been to india seven times in the last three years," blackman said, sipping a cup of chai, his cheeks flushed from canvassing in the cold, ahead of britain\'s first december election in nearly a century',
 ' "i am well-known for being a very pro-indian supporter',
 '"across the table, kuldeep singh shekhawat, the head of a uk-based pressure group linked to modi\'s bharatiya janata party (bjp), chimed in: "a hundred percent',
 '" blackman is a three-term member of parliament for harrow east, a racially and religiously diverse constituency where 28',
 '2% of the residents are hindu',
 ' the conservative party politician has a track record of championing ind

In [0]:
nlp.vocab['cnn'].is_stop = True
nlp.Defaults.stop_words |= {'cnn', ''}
def tokenize(text):
  doc = nlp(text)
  tokens = [token.text 
            for token in doc 
            if not token.is_stop and 
               not token.is_punct and
               token.pos_ == 'NOUN']
  return tokens
for i, text in enumerate(test):
  test[i] = tokenize(text)

In [0]:
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora
import re

In [0]:
id2word = corpora.Dictionary(test)

id2word.filter_extremes(no_below=5, no_above=0.9)
corpus = [id2word.doc2bow(text) for text in test]
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics = 5,
                   passes=10,
                   workers=4)

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [11]:
topics[0]

'blackman indians group kashmir minorities'

# Sentiment Analysis

In [0]:
from textblob import TextBlob
def get_sentiment(text): 
    analysis = TextBlob(text) 

    if analysis.sentiment.polarity > 0: 
        return 'positive', analysis
    elif analysis.sentiment.polarity == 0: 
        return 'neutral', analysis.sentiment.polarity
    else: 
        return 'negative', analysis.sentiment.polarity

In [0]:
_, temp = get_sentiment(get_text('https://www.cnn.com/2019/11/22/us/trump-administration-seizing-border-wall-land/index.html'))

In [14]:
temp.sentiment

Sentiment(polarity=0.0968393639300719, subjectivity=0.42498643195545843)

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Summarize

In [16]:
article = get_text('http://rss.cnn.com/~r/rss/cnn_topstories/~3/CO9DFp_HIA4/index.html')
article = article.replace('\n', ' ')
article = article.lower().split('.')
article

['on a recent november evening, bob blackman scrolled through his phone in a nondescript indian restaurant in west london, sparsely decorated except for stock pictures of east african sunsets and lions',
 " he proudly pointed to his phone's background photo, which shows him smiling next to indian prime minister narendra modi",
 '"i\'ve been to india seven times in the last three years," blackman said, sipping a cup of chai, his cheeks flushed from canvassing in the cold, ahead of britain\'s first december election in nearly a century',
 ' "i am well-known for being a very pro-indian supporter',
 '"across the table, kuldeep singh shekhawat, the head of a uk-based pressure group linked to modi\'s bharatiya janata party (bjp), chimed in: "a hundred percent',
 '" blackman is a three-term member of parliament for harrow east, a racially and religiously diverse constituency where 28',
 '2% of the residents are hindu',
 ' the conservative party politician has a track record of championing ind

In [0]:
formatted_article = article[::]
for i, text in enumerate(formatted_article):
  formatted_article[i] = tokenize(text)

In [0]:
word_frequencies = {}
for sentence in formatted_article:
  for word in sentence:
    if word not in word_frequencies.keys():
      word_frequencies[word] = 1
    else:
      word_frequencies[word] += 1

In [0]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

In [0]:
sentence_scores = {}
for i, sentence in enumerate(article):
  for word in formatted_article[i]:
    if word in word_frequencies.keys():
      if sentence not in sentence_scores.keys():
        sentence_scores[sentence] = word_frequencies[word]
      else:
        sentence_scores[sentence] += word_frequencies[word]

In [21]:
import heapq
summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)
summary_sentences

[' india\'s modi made the election a referendum on his leadership -- and it paid off suresh grover, a human rights campaigner and director of the monitoring group, said ofbjp branches were established in the us and uk to "regularly discuss how to popularize bjp policies overseas, including changing the uk\'s perception of modi himself',
 '" earlier this month, labour revised its position on kashmir, announcing in a letter from the party\'s chairman that the issue was "a bilateral matter for india and pakistan" to solve and warning that the issue "must not be allowed to divide communities against each other here in the uk',
 'must watchwhat\'s at stake in britain\'s volatile election 03:00"they [the labour party] did it with the jewish community, and it is now doing it with the indian community: taking loyal, solid voters for granted," meghani told cnn, referring to criticism over labour\'s handling of a recent wave of anti-semitic incidents within the party',
 ' "your views about an is

# Completed Output

In [0]:
!pip install feedparser

In [0]:
import feedparser
from bs4 import BeautifulSoup
import requests
import heapq
import spacy
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora
import re
from textblob import TextBlob
import nltk
import en_core_web_sm
nltk.download('punkt')
nlp = en_core_web_sm.load()

In [0]:
def read_feed(rss_url):
  try:
    rss_feed = feedparser.parse(rss_url)
    return rss_feed
  except:
    return None

def parse_rss_feed(rss_feed):
  parsed_feed = []
  
  for entry in rss_feed.entries:
    parsed_feed.append({
        'title': entry.title,
        'url': entry.link,
        'description': entry.description,
        'summary': None,
        'sentiment': None,
        'topics': None
    })

  return parsed_feed

def replacements(text):
  stop_phrases = ['Read More', '\n', "\\'s"]
  
  for item in stop_phrases:
    text = text.replace(item, ' ')
  
  return text

def get_text(url):
  html = requests.get(url)
  soup = BeautifulSoup(html.text)

  # Strip out any excess html
  for script in soup(["script", "style"]):
      script.decompose()

  text = soup.find('section', {'id':'body-text'}).get_text() # CNN specific
  text = replacements(text)
  start = text.find('(CNN)') # CNN specific
  end = text.find('\nSearch') # CNN specific
  return text[start+5:end]

def get_sentiment(text): 
    analysis = TextBlob(text) 

    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

def sentence_split(text):
  text = text.lower().split('.')
  return text

# Tags
# Summary
# Combine

In [48]:
get_text('http://rss.cnn.com/~r/rss/cnn_topstories/~3/CO9DFp_HIA4/index.html')

'On a recent November evening, Bob Blackman scrolled through his phone in a nondescript Indian restaurant in west London, sparsely decorated except for stock pictures of East African sunsets and lions. He proudly pointed to his phone\'s background photo, which shows him smiling next to Indian Prime Minister Narendra Modi."I\'ve been to India seven times in the last three years," Blackman said, sipping a cup of chai, his cheeks flushed from canvassing in the cold, ahead of Britain\'s first December election in nearly a century. "I am well-known for being a very pro-Indian supporter."Across the table, Kuldeep Singh Shekhawat, the head of a UK-based pressure group linked to Modi\'s Bharatiya Janata Party (BJP), chimed in: "A hundred percent." Blackman is a three-term member of Parliament for Harrow East, a racially and religiously diverse constituency where 28.2% of the residents are Hindu. The Conservative Party politician has a track record of championing Indian -- more specifically, Hi